In [1]:
import cv2

cam = cv2.VideoCapture(0)

while cam.isOpened():
    ret, frame = cam.read()
    if not ret:
        break

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    CHESS_BOARD_DIM = (6, 4)


    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Encontrar las esquinas del tablero de ajedrez
    ret, corners = cv2.findChessboardCorners(gray, CHESS_BOARD_DIM)

    # Si son encontradas, refinar las esquinas y diujarlas en la imagen a color
    if ret:
        corners1 = cv2.cornerSubPix(gray, corners, (11, 11), (-1,-1), criteria)
        cv2.drawChessboardCorners(frame, CHESS_BOARD_DIM, corners1, ret)

    cv2.imshow('Ejercicio 1', frame)
    key = cv2.waitKey(1)
    if key == ord('q'):
        break

cam.release()
cv2.destroyAllWindows()

In [3]:
import os
import numpy as np

CHESS_BOARD_DIM = (6, 4)
SQUARE_SIZE = 20
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

calib_data_path = "calib_data"

CHECK_DIR = os.path.isdir(calib_data_path)
if not CHECK_DIR:
    os.mkdir(calib_data_path)
    print(f"Directorio {calib_data_path} creado.")

else:
    print(f"Directorio {calib_data_path} ya existe.")

obj_3D = np.zeros((CHESS_BOARD_DIM[0] * CHESS_BOARD_DIM[1], 3), np.float32)
obj_3D[:, :2] = np.mgrid[0:CHESS_BOARD_DIM[0], 0:CHESS_BOARD_DIM[1]].T.reshape(-1, 2)
obj_3D *= SQUARE_SIZE

obj_points_3D = []
img_points_2D = []

image_dir_path = "Fotos"

files = os.listdir(image_dir_path)
for file in files:
    print(file)
    imagePath = os.path.join(image_dir_path, file)

    image = cv2.imread(imagePath)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, CHESS_BOARD_DIM, None)
    if ret:
        obj_points_3D.append(obj_3D)
        corners1 = cv2.cornerSubPix(gray, corners, (3, 3), (-1, -1), criteria)
        img_points_2D.append(corners1)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(obj_points_3D, img_points_2D, gray.shape[::-1], None, None)
print("Calibrada")

print("Guardando datos de calibración")
np.savez(
    f"{calib_data_path}/MulMatrix",
    camMatrix=mtx,
    distCoef=dist,
    rVector=rvecs,
    tVector=tvecs,
)

Directorio calib_data ya existe.
1.png
10.png
2.png
3.png
4.png
5.png
6.png
7.png
8.png
9.png
Calibrada
Guardando datos de calibración


In [4]:
calib_data_path = 'calib_data/MulMatrix.npz'
calib_data = np.load(calib_data_path)

cam_mat = calib_data['camMatrix']
dist_coef = calib_data['distCoef']
r_vectors = calib_data['rVector']
t_vectors = calib_data['tVector']

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    frame_undist = frame.copy()
    if ret:
        cv2.imshow('Camara con distorcion', frame_undist)
        h, w = frame.shape[:2]
        newcammat, roi = cv2.getOptimalNewCameraMatrix(cam_mat, dist_coef, (w, h), 1, (w, h))
        dst = cv2.undistort(frame, cam_mat, dist_coef, None, newcammat)

        x, y, w, h = roi
        dst = dst[y:y+h, x:x+w]

        cv2.imshow('Camara sin distorcion', dst)

        key = cv2.waitKey(1)
        if key == ord("q"):
            break

cap.release()
cv2.destroyAllWindows()